# 📊 Automated Report Export & Notification
This notebook does the following:
1. Reads Unity Catalog views
2. Exports them to CSV (on DBFS)
3. Sends notifications (to be scheduled via job)
4. Adds manual guidance to link into dashboards

In [0]:
import json
import requests
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Path to metadata file (DBFS or ADLS)
metadata_path = "abfss://kyc-data@reprotingfactorydl.dfs.core.windows.net/finance/kyc/metadata/report_definitions_prompt.json"

# Load metadata
try:
    metadata_df = spark.read.option("multiline", "true").json(metadata_path)
    reports = metadata_df.toJSON().map(json.loads).collect()
except Exception as e:
    raise Exception(f"Failed to load metadata file: {metadata_path}. Error: {e}")

In [0]:
# Simulated email or Slack notification (replace with real APIs as applicable)
def notify_users(report_name, channel="email"):
    if channel == "slack":
        print(f"[Slack] Report '{report_name}' is now available.")
    else:
        print(f"[Email] Report '{report_name}' generated and certified.")

In [0]:
# Define parameters
report_views = [
    "finance.kyc_ml.fraud_summary_by_account_type",
    "finance.kyc_ml.high_risk_summary"
]
export_base_path = "abfss://kyc-data@reprotingfactorydl.dfs.core.windows.net/finance/kyc/reports/"
export_format = "csv"  # or 'parquet' or 'json'
send_email_to = ["rajeevjha@aol.in"]  # Placeholder

In [0]:
# Export all views to DBFS
for view in report_views:
    df = spark.table(view)
    name = view.split(".")[-1]
    export_path = f"{export_base_path}{name}.{export_format}"
    df.write.mode("overwrite").option("header", True).format(export_format).save(export_path)
    print(f"Exported {view} to {export_path}")

## 📬 Placeholder for Email Notification
_Note: You can use Databricks Jobs email notification, or integrate with SMTP using `smtplib` in a secure job._

In [0]:
# Loop through reports to simulate visual and notification steps
for report in reports:
    try:
        print(f"Dashboard tile for {report['report_name']} ready!")
        if report.get("certify", False):
            notify_users(report['report_name'], channel="email")
    except Exception as e:
        print(f"Error setting up dashboard for {report['report_name']}: {e}")

## 📊 Manual Dashboard Creation Guide
1. Open Databricks SQL → Create Dashboard
2. Use above views as SQL sources (search by name)
3. Create charts: bar, pie, KPI, etc.
4. Save dashboard and schedule refresh/email

**Tip**: Use `lineage.refresh_time` or tags to filter latest views